In [134]:
import numpy as np
import glob, os
from pathlib import Path
import librosa
import scipy
import shutil
import gzip

## Function Definitions

In [193]:

def load_folder(data_path,max_count):
    ## Function to load .wav files in a given folder to an array of (data, sample rate) ##
    # INPUTS:
    # data_path = string to folder containing files to be loaded
    # max_count = number of files you wish to load in
        ## max_count can be used to only load the first "max_count" number of files from a folder,
        # if max_count == 0 then whole folder will be loaded
    ## ------------------------------ 
  
    samples = [];
    count = 0;
    
    if max_count!= 0: #Load first 'max_count' number of files from folder
        for file in glob.glob(os.path.join(data_path,'*.wav')):
            if count < max_count:
                temp,sr = librosa.load(file);
                temp = librosa.util.fix_length(temp,2*sr);
                samples.append([temp,sr])
                count+=1;
                
    else:
    #load whole folder
         for file in glob.glob(os.path.join(data_path,'*.wav')):
                temp,sr = librosa.load(file);
                temp = librosa.util.fix_length(temp,2*sr);
                samples.append([temp,sr])
                
    return samples
            
    
def nsy_sep(src_path,dest_path): 
    
    ## Specific function to separate nsynth dataset into folders based on instument family, returns nothing ##
    # INPUTS:
    # src_path = path to folder containing all nsynth files with no separation
    # dest_path = destination path for folder containing premade folders for each instrument family labeled after
    # the first three letters in the family name (ie, bass = bas, brass = bra, vocals = voc etc.)
    ## -------------------------
    for file in glob.glob(os.path.join(src_path,'*.wav')):
        fam = str.split(file,src_path+'\\')[1][:3];
        folder = os.path.join(dest_path,fam);
        shutil.move(file,folder);
        
        

def mel_spec_it(signal,fs):
    ## Function to find Mel Spectrogram of a signal ##
    # INPUTS:
    # signal
    # fs = sample rate
    # Returns an array of shape(n_mel,time)
    ##--------------------------    
    melS = librosa.feature.melspectrogram(signal,fs) # Mel Spectorgram
  
    return melS

def load_npz(npz_file):
    ## Function to load a compressed .npz file into an array ##
    ## -------------
    aloda = np.load(npz_file)
    dest_arry = aloda['a']
    
    return dest_arry

    

## Separating Nsynth Instrument Families 

In [149]:
src = 'D:\\SD Project\\nsynth\\nsynth-train.jsonwav.tar\\nsynth-train.jsonwav\\nsynth-train\\audio'
des = 'D:\\SD Project\\nsynth\\instr_families'

In [150]:
#uncomment to re-run separation
# nsy_sep(src,des) 



## Loading Partial Nsynth Sets and finding Mel Specs
(this takes a while so run carefully)

- save files to folder at the end

file contains array of shape(#sampes, n_mel, time)
 in the case of 1000 samples, 128 mel 

In [188]:
# Load in 1000 Samples of each instrument
path_bas = 'D:\\SD Project\\nsynth\\instr_families\\bas'
path_voc = 'D:\\SD Project\\nsynth\\instr_families\\voc'

bas_samp = load_folder(path_bas,1000)
voc_samp = load_folder(path_voc,1000)



Exception ignored in: <bound method ZipFile.__del__ of <zipfile.ZipFile file=<gzip on 0x241b3c7c198> mode='w'>>
Traceback (most recent call last):
  File "C:\Users\anna_\Anaconda3\lib\zipfile.py", line 1663, in __del__
    self.close()
  File "C:\Users\anna_\Anaconda3\lib\zipfile.py", line 1672, in close
    raise ValueError("Can't close the ZIP file while there is "
ValueError: Can't close the ZIP file while there is an open writing handle on it. Close the writing handle before closing the zip.


In [189]:
np.shape(bas_samp)

(1000, 2)

In [190]:
# Generate Mel Spectrograms
melSpecs_bas = [mel_spec_it(x[0],x[1]) for x in bas_samp]
melSpecs_voc = [mel_spec_it(x[0],x[1]) for x in voc_samp]

In [191]:
# Save Spectrograms to compressed files
f_mel_bas = 'D:\\SD Project\\nsynth\\instr_families\\MelSpecs_comp\\bas_1000.npz'
f_mel_voc = 'D:\\SD Project\\nsynth\\instr_families\\MelSpecs_comp\\voc_1000.npz'

np.savez_compressed(f_mel_bas,a = melSpecs_bas)
np.savez_compressed(f_mel_voc,a = melSpecs_voc)



## Load in .npz Files


In [204]:
bas_mel = load_npz(f_mel_bas)
np.shape(bas_mel)

(1000, 128, 87)

In [195]:
voc_mel = mel = load_npz(f_mel_voc)
np.shape(voc_mel)

(1000, 128, 87)

## Scratch Paper - IGNORE 


In [187]:
p = 'D:\\SD Project\\nsynth\\instr_families\\bas'
samp = load_folder(p,4);

comp_path = 'D:\\SD Project\\nsynth\\instr_families\\MelSpecs_comp'

np.shape(samp)



(4, 2)

In [165]:
# for x in samp:
    
#     melsp = mel_spec_it(x[0],x[1],comp_path)

melspecs =[mel_spec_it(x[0], x[1]) for x in samp]

In [166]:
np.shape(melspecs)

(2, 128, 87)

In [174]:
# Save Array as compressed file
f = 'D:\\SD Project\\nsynth\\instr_families\\MelSpecs_comp.npz'

np.savez_compressed(f,a = melspecs)


In [179]:
# extract/load compressed file back into array
aloda = np.load(f)

mel = aloda['a']

np.shape(mel)

# mix = np.vstack([bas_mel,voc_mel]) 
# np.shape(mix)


(2, 128, 87)